In [35]:
# Import the necessary libraries
from PIL import Image
import numpy as np
import os
from sklearn.decomposition import PCA
root_dir = os.getcwd()

src = os.path.join(root_dir,'YALE/centered/')
allFileNames = os.listdir(src)


# srccropped = os.path.join(root_dir,'YALE/aahancroped/')
# cropppedFileNames = os.listdir(src)

# load the image and convert into
# numpy array
numclasses = 15
picsperclass = 11
# picsperclass = 22
y = np.zeros(165)
numEigenfaces = 146 #previoulsy used 136
skippedeigenfaces = 0
newsize = (195, 231) # previously used  (160, 121)
# newsize = (195- 195//6, 231-231//6)
print(newsize)
origdata = np.zeros((newsize[0]*newsize[1],numclasses * picsperclass))
i = 0


for name in allFileNames:

    img = Image.open(src + name)
    img = img.convert("L")
    img = img.resize(newsize)
    if(i  == 4):
        img.show()
    y[i] = i//11
    numpydata = np.asarray(img)
    origdata[:,i] = numpydata.flatten()
    i+=1
    
    # img = Image.open(srccropped + cropppedFileNames[i//2])
    # img = img.convert("L")
    # img = img.resize(newsize)
    # if(i  == 5):
    #     img.show()
    # numpydata = np.asarray(img)
    # origdata[:,i] = numpydata.flatten()
    # i+=1

# Transposed Data
origdata = origdata.T
data = np.copy(origdata)
print("Data Shape: ", data.shape)
#image size is 218*178

#zero mean the data
mean = np.mean(data,axis = 0).reshape(-1,1)

# Reshape the data
data = data - mean.reshape(1,-1)

print("Mean Reshaped: ", mean.shape)

(195, 231)
Data Shape:  (165, 45045)
Mean Reshaped:  (45045, 1)


In [36]:
# X = data 
# pca = PCA(n_components=numEigenfaces)
# X_pca = pca.fit_transform(X)

In [37]:
# class_means = np.array([np.mean(X_pca[y == c], axis=0) for c in np.unique(y)])

# S_w = np.zeros((numEigenfaces,numEigenfaces))
# for c in np.unique(y):
#     X_c = X_pca[y == c]
#     class_mean = class_means[int(c)]
#     S_w += np.dot((X_c - class_mean).T, (X_c - class_mean))

# S_b = np.zeros((numEigenfaces,numEigenfaces))
# overall_mean = np.mean(X_pca, axis=0)
# # print(overall_mean) # this is zero
# for c in np.unique(y):
#     X_c = X_pca[y == c]
#     class_mean = class_means[int(c)]
#     n_c = X_c.shape[0]
#     S_b += n_c * np.outer((class_mean - overall_mean), (class_mean - overall_mean))


In [38]:
# from scipy.linalg import eig, eigh

# # geneigvals, geneigvecs = eigh(S_b, S_w)

# # geneigvals = geneigvals[::-1]
# # geneigvecs =  geneigvecs[:,::-1]
# # geneigvecs = geneigvecs.T

# geneigvals, geneigvecs = np.linalg.eigh(np.dot(np.linalg.inv(S_w), S_b))
# geneigvecs = geneigvecs[:, np.argsort(geneigvals)[::-1]]

# geneigvecs = geneigvecs.T
# # for i in range(geneigvecs.shape[0]):
#     geneigvecs[i] = geneigvecs[i]/np.linalg.norm(geneigvecs[i])

In [39]:
# print(geneigvals)
# # print(np.linalg.norm(geneigvecs,axis=1))
# # print(w)

# geneigvecs = geneigvecs[:14]


In [40]:
# w = np.dot(geneigvecs, pca.components_)
# w = np.dot(w, np.linalg.inv(pca.get_covariance()))
# w += mean.reshape(1,-1)
# for i in range(w.shape[0]):
#     w[i] = w[i]/np.linalg.norm(w[i])

In [41]:
# toshow = w[1] +mean.reshape(1,-1)
# toshow = toshow.reshape(newsize) 
# toshow= Image.fromarray(toshow.astype(np.uint8), mode='L')
# toshow.show()


In [42]:
import numpy as np
from sklearn.decomposition import PCA

def fisherface(images, labels, num_components=10):
    # Compute the PCA and transform the images
    pca = PCA(n_components=num_components)
    X_pca = pca.fit_transform(images)

    # Compute the class means
    class_means = np.zeros((int(np.max(labels)), num_components))
    for c in range(1, int(np.max(labels)) + 1):
        class_means[c-1] = np.mean(X_pca[labels == c], axis=0)

    # Compute the within-class scatter matrix
    S_w = np.zeros((num_components, num_components))
    for c in range(1, int(np.max(labels)) + 1):
        X_c = X_pca[labels == c]
        class_mean = class_means[c-1]
        S_w += np.dot((X_c - class_mean).T, (X_c - class_mean))

    # Compute the between-class scatter matrix
    overall_mean = np.mean(X_pca, axis=0)
    S_b = np.zeros((num_components, num_components))
    for c in range(1, int(np.max(labels)) + 1):
        n_c = np.sum(labels == c)
        class_mean = class_means[c-1]
        S_b += n_c * np.dot((class_mean - overall_mean).T, (class_mean - overall_mean))

    # Compute the Fisherfaces
    eigenvalues, eigenvectors = np.linalg.eigh(np.dot(np.linalg.inv(S_w), S_b))
    idx = eigenvalues.argsort()[::-1]
    eigenvectors = eigenvectors[:,idx]

    # Compute the weights for each image
    w = np.dot(X_pca, eigenvectors)

    # Return the PCA model, the Fisherfaces, and the weights
    return pca, eigenvectors, w


In [43]:
temp = fisherface(origdata,y,14)

In [44]:
print(temp[0].mean_)
FinalMat = temp[1] @ temp[0].components_

[247.52727273 247.64242424 247.82424242 ... 214.09090909 214.4969697
 215.06060606]


In [45]:
print(FinalMat)

[[ 0.00096044  0.00110314  0.00112525 ...  0.00301865  0.00322109
   0.00297676]
 [-0.00102947 -0.00096988 -0.00104757 ... -0.00245258 -0.00235902
  -0.00263344]
 [ 0.00041464  0.00028582  0.00031155 ...  0.0037104   0.00401237
   0.00441918]
 ...
 [ 0.00160973  0.00159404  0.00152593 ... -0.00523789 -0.00535284
  -0.00521381]
 [ 0.00141608  0.00136833  0.0014254  ...  0.00327388  0.00380013
   0.00385208]
 [-0.00250312 -0.00243241 -0.00264789 ...  0.00395386  0.00402876
   0.00349443]]


In [46]:
import csv

with open("MeanAndFaces.csv", "w", newline="") as csvfile:
    # Create a CSV writer object
    writer = csv.writer(csvfile)
    
    # Write the data to the CSV file
    writer.writerow(temp[0].mean_.flatten())
    for i in range(FinalMat.shape[0]):
        writer.writerow(FinalMat[i])